# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GNNTrainer** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from gnn import GNNTrainer
from movie_lens_loader import MovieLensLoader
from llm import PromptBertClassifier, VanillaBertClassifier, AddingEmbeddingsBertClassifierBase

from transformers import AutoConfig

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [2]:
MODEL_NAME = "google/bert_uncased_L-2_H-128_A-2"
MODEL_HIDDEN_SIZE = AutoConfig.from_pretrained(MODEL_NAME).hidden_size
SMALL_KGE_DIMENSION = 4
LARGE_KGE_DIMENSION = MODEL_HIDDEN_SIZE
KGE_DIMENSIONS = [SMALL_KGE_DIMENSION, LARGE_KGE_DIMENSION] # Output Dimension of the GNN Encoder.
EPOCHS = 20
BATCH_SIZE = 64

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-latest-small.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [3]:

movie_lens_loader = MovieLensLoader(kge_dimensions = KGE_DIMENSIONS)

In [4]:
movie_lens_loader.llm_df.head()

,mappedUserId,mappedMovieId,title,genres,prompt,split
0,0,0,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy...",user: 0[SEP]title: Toy Story (1995)[SEP]genres...,val
1,0,2,Grumpier Old Men (1995),"['Comedy', 'Romance']",user: 0[SEP]title: Grumpier Old Men (1995)[SEP...,test
2,0,5,Heat (1995),"['Action', 'Crime', 'Thriller']",user: 0[SEP]title: Heat (1995)[SEP]genres: ['A...,val
3,0,43,Seven (a.k.a. Se7en) (1995),"['Mystery', 'Thriller']",user: 0[SEP]title: Seven (a.k.a. Se7en) (1995)...,test
4,0,46,"Usual Suspects, The (1995)","['Crime', 'Mystery', 'Thriller']","user: 0[SEP]title: Usual Suspects, The (1995)[...",train


Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
gnn_trainer =    GNNTrainer(movie_lens_loader.data, kge_dimension = SMALL_KGE_DIMENSION)
gnn_trainer_large = GNNTrainer(movie_lens_loader.data, hidden_channels=MODEL_HIDDEN_SIZE, kge_dimension=MODEL_HIDDEN_SIZE)

loading pretrained model
Device: 'cpu'
loading pretrained model
Device: 'cpu'


We then train and validate the model on the link prediction task.

If the model is already trained, we can skip this part.
Training the models can take up to 5 minutes.

In [6]:

print(gnn_trainer.kge_dimension)
#gnn_trainer.train_model(movie_lens_loader.gnn_train_data, EPOCHS)
#gnn_trainer.validate_model(movie_lens_loader.gnn_test_data)
print(gnn_trainer_large.kge_dimension)
#gnn_trainer_large.train_model(movie_lens_loader.gnn_train_data, EPOCHS)
#gnn_trainer_large.validate_model(movie_lens_loader.gnn_test_data)


4
128


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

In [7]:
gnn_trainer.get_embeddings(movie_lens_loader)
gnn_trainer_large.get_embeddings(movie_lens_loader)
movie_lens_loader.llm_df.head()

,mappedUserId,mappedMovieId,title,genres,prompt,split,user_embedding_4,movie_embedding_4,user_embedding_128,movie_embedding_128
0,0,0,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy...",user: 0[SEP]title: Toy Story (1995)[SEP]genres...,val,"[0.26242750883102417, 0.6281917691230774, -1.9...","[0.5263428688049316, 0.8551913499832153, -1.04...","[0.1969558298587799, -0.3648209869861603, 0.48...","[0.2373889982700348, 0.11266285181045532, 0.37..."
1,0,2,Grumpier Old Men (1995),"['Comedy', 'Romance']",user: 0[SEP]title: Grumpier Old Men (1995)[SEP...,test,"[0.2656306028366089, 0.4019412696361542, -2.16...","[1.1155420541763306, -0.5623853206634521, 0.05...","[0.23769348859786987, -0.36578071117401123, 0....","[-0.23246431350708008, 0.19648301601409912, -0..."
2,0,5,Heat (1995),"['Action', 'Crime', 'Thriller']",user: 0[SEP]title: Heat (1995)[SEP]genres: ['A...,val,"[0.30835822224617004, 0.4791635274887085, -1.9...","[1.2811049222946167, -0.16243532299995422, -1....","[0.23082172870635986, -0.30352190136909485, 0....","[-0.09493206441402435, -0.08772756159305573, -..."
3,0,43,Seven (a.k.a. Se7en) (1995),"['Mystery', 'Thriller']",user: 0[SEP]title: Seven (a.k.a. Se7en) (1995)...,test,"[0.30060991644859314, 0.48889434337615967, -2....","[0.4733828902244568, 0.272816926240921, -0.855...","[0.2240385115146637, -0.4613000750541687, 0.66...","[-0.07982495427131653, 0.03565080836415291, -0..."
4,0,46,"Usual Suspects, The (1995)","['Crime', 'Mystery', 'Thriller']","user: 0[SEP]title: Usual Suspects, The (1995)[...",train,"[0.12415044009685516, 0.4077088236808777, -2.2...","[0.23878994584083557, -0.15946447849273682, -0...","[0.1535215526819229, -0.4172874093055725, 0.60...","[-0.004737831652164459, -0.05886310338973999, ..."


Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [8]:
vanilla_bert_classifier = VanillaBertClassifier(movie_lens_loader.llm_df, batch_size=BATCH_SIZE, model_name=MODEL_NAME)

Next we generate a vanilla llm dataset and tokenize it for training.

In [9]:
dataset_vanilla = movie_lens_loader.generate_vanilla_dataset(vanilla_bert_classifier.tokenize_function)

Next we train the model on the produced dataset. This can be skipped, if already trained ones.

In [10]:
#vanilla_bert_classifier.train_model_on_data(dataset_vanilla, epochs=EPOCHS)

Next we initialize the prompt encoder only classifier. This classifier uses the vanilla prompt and the KGEs for its link prediction.

In [11]:
prompt_bert_classifier = PromptBertClassifier(movie_lens_loader, gnn_trainer.get_embedding, kge_dimension=SMALL_KGE_DIMENSION, batch_size=BATCH_SIZE, model_max_length = 512, model_name=MODEL_NAME, force_recompute=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We also generate a prompt dataset, this time the prompts also include the KGEs.

In [12]:
dataset_prompt = movie_lens_loader.generate_prompt_embedding_dataset(prompt_bert_classifier.tokenize_function, kge_dimension = prompt_bert_classifier.kge_dimension)

We also train the model. This can be skipped if already done ones.

In [13]:
#prompt_bert_classifier.train_model_on_data(dataset_prompt, epochs = EPOCHS)

In [14]:
adding_embedding_bert_only_classifier = AddingEmbeddingsBertClassifierBase(movie_lens_loader, gnn_trainer_large.get_embedding, kge_dimension=MODEL_HIDDEN_SIZE, batch_size=BATCH_SIZE, model_name=MODEL_NAME)
dataset_adding_embedding = movie_lens_loader.generate_adding_embedding_dataset(adding_embedding_bert_only_classifier.tokenizer.sep_token, adding_embedding_bert_only_classifier.tokenizer.pad_token, adding_embedding_bert_only_classifier.tokenize_function, kge_dimension = MODEL_HIDDEN_SIZE)

In [15]:
#adding_embedding_bert_only_classifier.train_model_on_data(dataset_adding_embedding, epochs = EPOCHS)

In [16]:
vanilla_bert_classifier.forward_dataset_and_save_outputs(dataset_vanilla, force_recompute=True)
prompt_bert_classifier.forward_dataset_and_save_outputs(dataset_prompt, force_recompute=True)
df = adding_embedding_bert_only_classifier.forward_dataset_and_save_outputs(dataset_adding_embedding, force_recompute=True)

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Vanilla train Forward Epoch 1 from 3
Vanilla train Forward Epoch 2 from 3
Vanilla train Forward Epoch 3 from 3
Vanilla test Forward Epoch 1 from 3
Vanilla test Forward Epoch 2 from 3
Vanilla test Forward Epoch 3 from 3
Vanilla val Forward Epoch 1 from 3
Vanilla val Forward Epoch 2 from 3
Vanilla val Forward Epoch 3 from 3
Prompt train Forward Epoch 1 from 3
Prompt train Forward Epoch 2 from 3
Prompt train Forward Epoch 3 from 3
Prompt test Forward Epoch 1 from 3
Prompt test Forward Epoch 2 from 3
Prompt test Forward Epoch 3 from 3
Prompt val Forward Epoch 1 from 3
Prompt val Forward Epoch 2 from 3
Prompt val Forward Epoch 3 from 3
Adding train Forward Epoch 1 from 3
Adding train Forward Epoch 2 from 3
Adding train Forward Epoch 3 from 3
Adding test Forward Epoch 1 from 3
Adding test Forward Epoch 2 from 3
Adding test Forward Epoch 3 from 3
Adding val Forward Epoch 1 from 3
Adding val Forward Epoch 2 from 3
Adding val Forward Epoch 3 from 3
